In [ ]:
from abc import ABC, abstractmethod

# Abstract Base Class
class Vehicle(ABC):
    def __init__(self, brand, model, year):
        self._brand = brand
        self._model = model
        self._year = year
        self._speed = 0  # Default speed is 0

    # Getter Methods
    def get_brand(self):
        return self._brand

    def get_model(self):
        return self._model

    def get_year(self):
        return self._year

    def get_speed(self):
        return self._speed

    # Setter Methods
    def set_brand(self, brand):
        self._brand = brand

    def set_model(self, model):
        self._model = model

    def set_year(self, year):
        self._year = year

    def set_speed(self, speed):
        self._speed = max(0, speed)  # Ensure speed is non-negative

    @abstractmethod
    def accelerate(self):
        pass

    @abstractmethod
    def brake(self):
        pass

    def display_info(self):
        print(f"Brand: {self._brand}, Model: {self._model}, Year: {self._year}, Speed: {self._speed}")

# Derived Class: Car
class Car(Vehicle):
    def __init__(self, brand, model, year, fuel_type):
        super().__init__(brand, model, year)
        self._fuel_type = fuel_type  # Private attribute

    def get_fuel_type(self):
        return self._fuel_type

    def set_fuel_type(self, fuel_type):
        self._fuel_type = fuel_type

    def accelerate(self):
        self._speed += 10
        print(f"Car accelerated! New speed: {self._speed}")

    def brake(self):
        self._speed = max(0, self._speed - 5)
        print(f"Car slowed down. New speed: {self._speed}")

# Derived Class: Bike
class Bike(Vehicle):
    def __init__(self, brand, model, year, has_gear):
        super().__init__(brand, model, year)
        self._has_gear = has_gear  # Private attribute

    def get_has_gear(self):
        return self._has_gear

    def set_has_gear(self, has_gear):
        self._has_gear = has_gear

    def accelerate(self):
        self._speed += 5
        print(f"Bike accelerated! New speed: {self._speed}")

    def brake(self):
        self._speed = max(0, self._speed - 3)
        print(f"Bike slowed down. New speed: {self._speed}")

# Demonstration of Polymorphism
car = Car("Toyota", "Camry", 2022, "Petrol")
bike = Bike("Yamaha", "MT-15", 2023, True)

print("\n--- Car Details ---")
car.display_info()
car.accelerate()
car.brake()

print("\n--- Bike Details ---")
bike.display_info()
bike.accelerate()
bike.brake()



--- Car Details ---
Brand: Toyota, Model: Camry, Year: 2022, Speed: 0
Car accelerated! New speed: 10
Car slowed down. New speed: 5

--- Bike Details ---
Brand: Yamaha, Model: MT-15, Year: 2023, Speed: 0
Bike accelerated! New speed: 5
Bike slowed down. New speed: 2


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style= "darkgrid", color_codes = True)
import warnings
warnings.filterwarnings('ignore')


In [4]:
# Import Dataset
data= pd.read_csv("loanPredict.csv")
data.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [8]:
data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [9]:
data.describe().T


,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,5403.459283,6109.041673,150.0,2877.5,3812.5,5795.00,81000.0
CoapplicantIncome,614.0,1621.245798,2926.248369,0.0,0.0,1188.5,2297.25,41667.0
LoanAmount,592.0,146.412162,85.587325,9.0,100.0,128.0,168.00,700.0
Loan_Amount_Term,600.0,342.000000,65.120410,12.0,360.0,360.0,360.00,480.0
Credit_History,564.0,0.842199,0.364878,0.0,1.0,1.0,1.00,1.0


Data Cleaning

In [11]:
data.shape

(614, 13)

In [12]:
data=data.drop_duplicates()

In [13]:
data.shape

(614, 13)

In [14]:
data=data.drop("Loan_ID", axis=1)

In [15]:
data.shape


(614, 12)

In [16]:
data.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [17]:
data.head(5)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [18]:
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed','Property_Area','Credit_History']
continuous_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']